# Pengumpulan Data

In [ ]:
pip install tweepy

In [ ]:
import tweepy

In [ ]:
auth = tweepy.OAuthHandler('dkiMQUzjsgWFAi3tDLcbQEfBL', '0hhKDffKwf58kXG5rPL8lMhKhNydOM6z1TSSeyDSrDJEtvKfYv')
auth.set_access_token('1191624070514671616-X8MqSDJKq1cWBcSPNu15GBPHGSSNUq', 'JNc9ajiXZZmY26Yh1jl8idWkuVl1YiTwGPQ3p6YoZHs2n')

api = tweepy.API(auth)

In [ ]:
# import json
kataKunci = '#belajardirumah -filter:retweets'

tweets = api.search(kataKunci, lang='id', tweet_mode='extended')

# Wadah kosong
id = []
text = []
created_at = []
username = []

for data in tweets:
    # print(json.dumps(data._json, indent=4))
    id.append( data._json['id'] )
    text.append( data._json['full_text'] )
    created_at.append( data._json['created_at'] )
    username.append( data._json['user']['screen_name'] )

In [ ]:
pip install openpyxl

In [ ]:
import pandas

dataFrame = pandas.DataFrame({
    'id': id,
    'tweet': text,
    'dibuat pada': created_at,
    'user': username
})

dataFrame

# dataFrame.to_excel('crawling_09-05-2021.xlsx', index=False)

# Pra-Pemrosesan

In [ ]:
pip install sastrawi

In [ ]:
pip install xlrd

In [ ]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
dataFrameSlangword = pandas.read_excel('tbl_slangword -- 1725.xls')

# dataFrameSlangword = dataFrameSlangword.append({'slangword': 'lho', 'kata_asli': 'loh'}, ignore_index=True)

dataFrameSlangword

### Contoh

In [ ]:
# text = 'saya pergi ke apotik utk beli obat'
text = 'abis mandi ku trus tidur'

for index in range(len(dataFrameSlangword['slangword'])):
    if dataFrameSlangword['slangword'][index] in text:
        text = re.sub(r'\b{}\b'.format(dataFrameSlangword['slangword'][index]), dataFrameSlangword['kata_asli'][index], text)

text

### End Contoh

In [ ]:
dataFrameStopword = pandas.read_excel('stopword - 209.xls')

# dataFrameStopword = dataFrameSlangword.append({'stopword': 'loh'}, ignore_index=True)

dataFrameStopword

### Contoh

In [ ]:
text = 'saya pergi ke apotek untuk beli obat'
# text = 'habis mandi ku terus tidur'

for index in range(len(dataFrameStopword['stopword'])):
    if dataFrameStopword['stopword'][index] in text:
        text = re.sub(r'\b{}\b'.format(dataFrameStopword['stopword'][index]), '', text)

text

### End Contoh

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

### Contoh

In [ ]:
# text = 'saya membeli obat ke apotek'
text = 'saya membantu membersihkan rumah'

text = stemmer.stem(text)

text

### End Contoh

In [ ]:
# dataFrame['tweet']

cleanText = []

for data in dataFrame['tweet']:
    
    # Casefolding
    resultText = data.lower()
    
    # Cleansing : Hapus URL, Mention, Hastag & Selain Huruf / Menghilangkan kata yang diawali dengan kata 'http', '@', '#' & selain huruf
    resultText = re.sub(r'http\S+', ' ', resultText) # hapus url
    resultText = re.sub(r'@\S+', ' ', resultText) # hapus mention
    resultText = re.sub(r'#\S+', ' ', resultText) # hapus hashtag
    resultText = re.sub(r'[^a-z ]', ' ', resultText) # hapus selain huruf a-z
    
    # Remove Whitespaces : Menghilangkan spasi/tab/baris yang kosong
    resultText = resultText.strip()
    resultText = re.sub('\s+', ' ', resultText)
    
    # Mengubah slangword
    for index in range(len(dataFrameSlangword['slangword'])):
        if dataFrameSlangword['slangword'][index] in resultText:
            resultText = re.sub(r'\b{}\b'.format(dataFrameSlangword['slangword'][index]), dataFrameSlangword['kata_asli'][index], resultText)
    
    # Menghapus slangword
    for index in range(len(dataFrameStopword['stopword'])):
        if dataFrameStopword['stopword'][index] in resultText:
            resultText = re.sub(r'\b{}\b'.format(dataFrameStopword['stopword'][index]), '', resultText)
    
    # Stemming (sastrawi)
    resultText = stemmer.stem(resultText)
    
    cleanText.append(resultText)

cleanText

In [ ]:
dataFrame['clean text'] = cleanText

dataFrame

# dataFrame.to_excel('preprocessing_09-05-2021.xlsx', index=False)